In [5]:
import csv
import os
from collections import defaultdict

def remove_high_null_columns_streaming(input_file, output_file, threshold=0.5, delimiter='\t'):
    """
    Remove columns with >threshold% null values using minimal memory.
    Reads file line-by-line without loading into memory.
    
    Parameters:
    - input_file: path to input file
    - output_file: path to output file  
    - threshold: fraction of nulls above which to remove column (default 0.5)
    - delimiter: '\t' for TSV, ',' for CSV
    """
    
    print("🔍 Starting memory-efficient analysis...")
    print(f"📁 Input: {input_file}")
    print(f"💾 Output: {output_file}")
    
    # Get file size for progress tracking
    file_size = os.path.getsize(input_file)
    print(f"📊 File size: {file_size / (1024**3):.2f} GB")
    
    # First pass: Count nulls and total rows
    null_counts = defaultdict(int)
    total_rows = 0
    column_names = []
    
    print("\n🔄 Pass 1: Counting null values...")
    
    try:
        with open(input_file, 'r', encoding='utf-8', errors='ignore') as f:
            reader = csv.reader(f, delimiter=delimiter)
            
            # Read header
            try:
                header = next(reader)
                column_names = header
                num_cols = len(column_names)
                print(f"📋 Found {num_cols} columns")
                
                # Initialize null counters
                for col in column_names:
                    null_counts[col] = 0
                    
            except StopIteration:
                print("❌ File is empty or invalid")
                return False
            
            # Process each row
            bytes_read = len(','.join(header).encode('utf-8'))
            
            for row_num, row in enumerate(reader, 1):
                total_rows += 1
                
                # Pad row if it's shorter than header
                while len(row) < num_cols:
                    row.append('')
                
                # Count nulls (empty strings, 'NULL', 'NaN', etc.)
                for i, value in enumerate(row[:num_cols]):
                    if not value or value.strip().lower() in ['', 'null', 'nan', 'none', 'na']:
                        if i < len(column_names):
                            null_counts[column_names[i]] += 1
                
                # Progress update every 50k rows
                if row_num % 50000 == 0:
                    bytes_read += sum(len(str(cell).encode('utf-8', errors='ignore')) for cell in row)
                    progress = (bytes_read / file_size) * 100
                    print(f"   📈 Processed {row_num:,} rows (~{progress:.1f}% of file)")
            
            print(f"✅ Pass 1 complete: {total_rows:,} rows analyzed")
            
    except Exception as e:
        print(f"❌ Error in pass 1: {e}")
        return False
    
    # Calculate which columns to keep
    print(f"\n🧮 Calculating null percentages...")
    
    columns_to_keep = []
    columns_to_remove = []
    
    for col in column_names:
        null_pct = null_counts[col] / total_rows if total_rows > 0 else 0
        
        if null_pct <= threshold:
            columns_to_keep.append(col)
        else:
            columns_to_remove.append((col, null_pct))
    
    # Sort removed columns by null percentage (worst first)
    columns_to_remove.sort(key=lambda x: x[1], reverse=True)
    
    print(f"📊 Results:")
    print(f"   Total rows: {total_rows:,}")
    print(f"   Original columns: {len(column_names)}")
    print(f"   Columns to keep: {len(columns_to_keep)}")
    print(f"   Columns to remove: {len(columns_to_remove)}")
    
    if columns_to_remove:
        print(f"\n🗑️  Worst offenders (showing top 10):")
        for col, pct in columns_to_remove[:10]:
            print(f"   - {col}: {pct*100:.1f}% null")
        if len(columns_to_remove) > 10:
            print(f"   ... and {len(columns_to_remove)-10} more")
    
    # Get indices of columns to keep
    keep_indices = [i for i, col in enumerate(column_names) if col in columns_to_keep]
    
    # Second pass: Write cleaned file
    print(f"\n💾 Pass 2: Writing cleaned data...")
    
    try:
        with open(input_file, 'r', encoding='utf-8', errors='ignore') as infile, \
             open(output_file, 'w', encoding='utf-8', newline='') as outfile:
            
            reader = csv.reader(infile, delimiter=delimiter)
            writer = csv.writer(outfile, delimiter=',')  # Output as CSV
            
            # Write filtered header
            header = next(reader)
            filtered_header = [header[i] for i in keep_indices if i < len(header)]
            writer.writerow(filtered_header)
            
            # Write filtered rows
            rows_written = 0
            
            for row in reader:
                rows_written += 1
                
                # Pad row if necessary
                while len(row) < len(column_names):
                    row.append('')
                
                # Write only kept columns
                filtered_row = [row[i] if i < len(row) else '' for i in keep_indices]
                writer.writerow(filtered_row)
                
                # Progress update
                if rows_written % 100000 == 0:
                    print(f"   📝 Written {rows_written:,} rows")
            
            print(f"✅ Pass 2 complete: {rows_written:,} rows written")
            
    except Exception as e:
        print(f"❌ Error in pass 2: {e}")
        return False
    
    # Final summary
    output_size = os.path.getsize(output_file)
    reduction = ((file_size - output_size) / file_size) * 100
    
    print(f"\n🎉 SUCCESS!")
    print(f"   📁 Input: {file_size / (1024**2):.1f} MB")
    print(f"   📁 Output: {output_size / (1024**2):.1f} MB") 
    print(f"   📉 Size reduction: {reduction:.1f}%")
    print(f"   🗂️  Columns: {len(column_names)} → {len(columns_to_keep)}")
    print(f"   💾 Saved to: {output_file}")
    
    return True

def quick_file_check(filename):
    """Quick check of file format and size"""
    try:
        size_gb = os.path.getsize(filename) / (1024**3)
        
        # Check delimiter by reading first line
        with open(filename, 'r', encoding='utf-8', errors='ignore') as f:
            first_line = f.readline()
            
        tabs = first_line.count('\t')
        commas = first_line.count(',')
        
        delimiter = '\t' if tabs > commas else ','
        format_type = 'TSV' if delimiter == '\t' else 'CSV'
        
        print(f"📋 File Info:")
        print(f"   Size: {size_gb:.2f} GB")
        print(f"   Format: {format_type}")
        print(f"   Delimiter: '{delimiter}'")
        
        return delimiter
        
    except Exception as e:
        print(f"❌ Error checking file: {e}")
        return None

# Usage
if __name__ == "__main__":
    # File paths - UPDATE THESE
    input_file = "/kaggle/input/bindingdb-all-202509-tsv/BindingDB_All.tsv"
    output_file = "BindingDB_cleaned.csv"
    
    print("🚀 Ultra Memory-Efficient Column Cleaner")
    print("=" * 50)
    
    # Check file format
    delimiter = quick_file_check(input_file)
    
    if delimiter:
        # Process the file
        success = remove_high_null_columns_streaming(
            input_file=input_file,
            output_file=output_file,
            threshold=0.95,  # Remove columns with >50% nulls
            delimiter=delimiter
        )
        
        if success:
            print("\n🎊 DONE! Your dataset has been cleaned successfully!")
            print(f"📂 Check your cleaned file: {output_file}")
        else:
            print("\n💥 Something went wrong. Check the error messages above.")
    else:
        print("❌ Could not process file. Check the file path and permissions.")

🚀 Ultra Memory-Efficient Column Cleaner
📋 File Info:
   Size: 6.19 GB
   Format: TSV
   Delimiter: '	'
🔍 Starting memory-efficient analysis...
📁 Input: /kaggle/input/bindingdb-all-202509-tsv/BindingDB_All.tsv
💾 Output: BindingDB_cleaned.csv
📊 File size: 6.19 GB

🔄 Pass 1: Counting null values...
📋 Found 640 columns
   📈 Processed 50,000 rows (~0.0% of file)
   📈 Processed 100,000 rows (~0.0% of file)
   📈 Processed 150,000 rows (~0.0% of file)
   📈 Processed 200,000 rows (~0.0% of file)
   📈 Processed 250,000 rows (~0.0% of file)
   📈 Processed 300,000 rows (~0.0% of file)
   📈 Processed 350,000 rows (~0.0% of file)
   📈 Processed 400,000 rows (~0.0% of file)
   📈 Processed 450,000 rows (~0.0% of file)
   📈 Processed 500,000 rows (~0.0% of file)
   📈 Processed 550,000 rows (~0.0% of file)
   📈 Processed 600,000 rows (~0.0% of file)
   📈 Processed 650,000 rows (~0.0% of file)
   📈 Processed 700,000 rows (~0.0% of file)
   📈 Processed 750,000 rows (~0.0% of file)
   📈 Processed 800,000 r

In [8]:
# show all rows (no truncation)
pd.set_option("display.max_rows", None)

# optional: show all columns if needed
pd.set_option("display.max_columns", None)

In [6]:
import pandas as pd 
df=pd.read_csv('/kaggle/working/BindingDB_cleaned.csv')

/tmp/ipykernel_77/1058850448.py:2: DtypeWarning: Columns (8,10,12,14,17,18,30,38,39,40,41) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/kaggle/working/BindingDB_cleaned.csv')


In [9]:
df.isnull().sum()

BindingDB Reactant_set_id                                                     0
Ligand SMILES                                                                11
Ligand InChI                                                             174513
Ligand InChI Key                                                         174513
BindingDB MonomerID                                                           0
BindingDB Ligand Name                                                         0
Target Name                                                                   0
Target Source Organism According to Curator or DataSource                947169
Ki (nM)                                                                 2449633
IC50 (nM)                                                               1002257
EC50 (nM)                                                               2782660
pH                                                                      2844019
Temp (C)                                